In [1]:
# # Install environment and agent
# !pip install highway-env
# # TODO: we use the bleeding edge version because the current stable version does not support the latest gym>=0.21 versions. Revert back to stable at the next SB3 release.
# !pip install git+https://github.com/DLR-RM/stable-baselines3

# # Environment
import gymnasium as gym
import highway_env

# Agent
from stable_baselines3 import DQN


import sys
# from tqdm.notebook import trange
# !pip install tensorboardx gym pyvirtualdisplay
# !apt-get install -y xvfb ffmpeg

2024-10-10 08:40:56.845085: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-10 08:40:56.893592: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-10-10 08:40:57.328790: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-10 08:40:59.706284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import base64
from pathlib import Path

from gymnasium.wrappers import RecordVideo
from IPython import display as ipythondisplay
from pyvirtualdisplay import Display



def record_videos(env, video_folder="videos"):
    wrapped = RecordVideo(
        env, video_folder=video_folder, episode_trigger=lambda e: True
    )

    # Capture intermediate frames
    env.unwrapped.set_record_video_wrapper(wrapped)

    return wrapped


def show_videos(path="videos"):
    html = []
    for mp4 in Path(path).glob("*.mp4"):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                      loop controls style="height: 400px;">
                      <source src="data:video/mp4;base64,{}" type="video/mp4" />
                 </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))


In [3]:
from stable_baselines3 import DQN
import pprint
from matplotlib import pyplot as plt
import numpy as np

llm stuff


In [4]:
!pip install -q transformers huggingface_hub[cli] bitsandbytes accelerate

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [ ]:
# Define Boto3 client for Bedrock
client = boto3.client("bedrock-runtime", region_name="us-east-1")
model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Claude API settings
CLAUDE_API_KEY = os.getenv('CLAUDE_API_KEY')
CLAUDE_API_URL = 'https://api.anthropic.com/v1/complete'


def claude_action(prompt1, assist1, prompt2, model='claude-v1', max_tokens_to_sample=50, temperature=0.7):
    """
    Sends prompts to Claude.ai and retrieves the recommended action.
    """
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {CLAUDE_API_KEY}',
    }

    full_prompt = f"{prompt1}\n\n{assist1}\n\n{prompt2}"
    
    native_request = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 1024,
        "temperature": 0,
        "messages": [
            {
                "role": "user",
                "content": [{"type": "text", "text": full_prompt}],
            }
        ],
    }
    request = json.dumps(native_request)
    try:
        response = client.invoke_model(modelId=model_id, body=request)
        model_response = json.loads(response["body"].read())
        response_json = model_response["content"][0]["text"]
        #action_text = response_json.get('completion', '').strip()
        if 'Final decision:' in response_json:
            action = response_json.split('Final decision:')[-1].strip().upper()
            return action
        else:
            print(f"Unexpected response format: {action_text}")
            return 'IDLE'
    except requests.exceptions.RequestException as e:
        print(f"Error communicating with Claude.ai: {e}")
        time.sleep(1)
        return 'IDLE'

In [10]:
action_dict = {
    0: 'LANE_LEFT',
    1: 'IDLE',
    2: 'LANE_RIGHT',
    3: 'FASTER',
    4: 'SLOWER'
}

In [15]:
def rf_query(obs):
    ##load model
    rf_model = joblib.load("models/random_forest_model.pkl")
    #print(f"Model loaded")
    ##
    obs_flat = obs.flatten().reshape(1, -1)  # Flatten and reshape observation
    return rf_model.predict(obs_flat)[0]

In [16]:
class MyHighwayEnvllm(gym.Env):

    def __init__(self, vehicleCount=10):
        super(MyHighwayEnvllm, self).__init__()
        # base setting
        self.vehicleCount = vehicleCount
        self.prev_action  = 'FASTER'

        # environment setting
        self.config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 2,
            "show_trajectories": True,
            "render_agent": True,
        }
        self.env = gym.make("highway-v0", config = self.config)
        self.env.configure(self.config)
        self.action_space = self.env.action_space
        self.observation_space = gym.spaces.Box(
            low=-np.inf,high=np.inf,shape=(10,5),dtype=np.float32
        )

    def find_smallest_positive(self, arr):
        smallest_positive = float('inf')
        index = -1

        for i, value in enumerate(arr):
            if 0 < value < smallest_positive:
                smallest_positive = value
                index = i

        return smallest_positive, index

    def step(self, action):

        # Step the wrapped environment and capture all returned values
        obs, dqn_reward, done, truncated, info = self.env.step(action)

        ##combine rewards randomly
        if np.random.rand() <= 0.33:

            llm_response = rf_query(obs)

            l_acts  = 0

            if llm_response is not None:
                l_acts += 1
                ##modify action accordingly
                llm_act = llm_response

            if l_acts == 1:
                if llm_act == action:
                    llm_reward = 1
                else:
                    llm_reward = 0
                comb_reward = 0.7*dqn_reward + 0.3*llm_reward
            else:
                comb_reward = dqn_reward

        else:
            comb_reward = dqn_reward

        self.prev_action = action_dict[action]

        Reward = 1 / (1 + np.exp(-comb_reward))

        return obs, Reward, done, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return obs  # Make sure to return the observation

In [17]:
from stable_baselines3.common.vec_env import DummyVecEnv
env_llm = MyHighwayEnvllm()

c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [18]:
model = DQN('MlpPolicy', env_llm,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            buffer_size=15000,
            learning_starts=200,
            batch_size=32,
            gamma=0.8,
            train_freq=1,
            gradient_steps=1,
            target_update_interval=50,
            exploration_fraction=0.7,
            verbose=1)

model.learn(int(2e4))
model.save('models/dqn_model_claude')
# Download the model to your local machine

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.5      |
|    ep_rew_mean      | 4.27     |
|    exploration_rate | 0.998    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 2        |
|    time_elapsed     | 10       |
|    total_timesteps  | 26       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.62     |
|    ep_rew_mean      | 4.38     |
|    exploration_rate | 0.996    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 2        |
|    time_elapsed     | 22       |
|    total_timesteps  | 53       |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 6.08     |
|    ep_rew_mean      | 3.98  

In [1]:
from gymnasium.wrappers import RecordVideo
# base setting
vehicleCount = 10

# environment setting
config = {

    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": True,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 2,
    "show_trajectories": True,
    "render_agent": True,
}

modelDqnllm = DQN.load("models/dqn_model")

env = gym.make('highway-v0', render_mode='rgb_array')
env.configure(config)
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done, truncated = env.reset(), False, False
    while not (done or truncated):
        action, _ = modelDqnllm.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()

NameError: name 'np' is not defined

Train dqn on roundabout

In [20]:
class MyHighwayEnv_roundabout(gym.Env):
    def __init__(self, vehicleCount=10):
        super(MyHighwayEnv_roundabout, self).__init__()
        # base setting
        self.vehicleCount = vehicleCount

        # environment setting
        self.config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": True,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 2,
            "show_trajectories": True,
            "render_agent": True,
        }
        self.env = gym.make("roundabout-v0")
        self.env.configure(self.config)
        self.action_space = self.env.action_space
        self.observation_space = gym.spaces.Box(
            low=-np.inf,high=np.inf,shape=(10,5),dtype=np.float32
        )

    def step(self, action):
        # Step the wrapped environment and capture all returned values
        obs, dqn_reward, done, truncated, info = self.env.step(action)
        ##reward shaping takes place
        Reward = 1 / (1 + np.exp(-dqn_reward))

        return obs, Reward, done, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return obs  # Make sure to return the observation

In [21]:
from stable_baselines3.common.vec_env import DummyVecEnv
env_roundabout = MyHighwayEnv_roundabout()

c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [22]:
model = DQN('MlpPolicy', env_roundabout,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            buffer_size=15000,
            learning_starts=200,
            batch_size=32,
            gamma=0.8,
            train_freq=1,
            gradient_steps=1,
            target_update_interval=50,
            exploration_fraction=0.7,
            verbose=1,
            tensorboard_log='highway_dqn/')

model.learn(int(2e4))
model.save('models/dqn_model_roundabout')
# Download the model to your local machine


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_11
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32.5     |
|    ep_rew_mean      | 23.3     |
|    exploration_rate | 0.991    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 15       |
|    time_elapsed     | 8        |
|    total_timesteps  | 130      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 32.4     |
|    ep_rew_mean      | 22.2     |
|    exploration_rate | 0.982    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 15       |
|    time_elapsed     | 16       |
|    total_timesteps  | 259      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.0324   |
|    n_updates        | 58       |


In [23]:
class MyRoundaboutEnv_llm(gym.Env):
    def __init__(self, vehicleCount=10):
        super(MyRoundaboutEnv_llm, self).__init__()
        self.vehicleCount = vehicleCount
        self.prev_action = 'FASTER'

        # Environment configuration
        self.config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 2,
            "show_trajectories": True,
            "render_agent": True,
        }
        self.env = gym.make("roundabout-v0")  # Change to the roundabout environment
        self.env.configure(self.config)
        self.action_space = self.env.action_space
        self.observation_space = gym.spaces.Box(
            low=-np.inf, high=np.inf, shape=(10, 5), dtype=np.float32
        )

    def find_smallest_positive(self, arr):
        smallest_positive = float('inf')
        index = -1
        for i, value in enumerate(arr):
            if 0 < value < smallest_positive:
                smallest_positive = value
                index = i
        return smallest_positive, index

    def prompt_design(self, obs_):
        prompt1 = 'You are a driving assistant navigating a roundabout. The ego vehicle is approaching a roundabout with moving traffic. You need to recommend ONLY ONE action from the following set, considering the current scenario:\n \
        \t1. IDLE -- maintain the current speed in the current lane \n \
        \t2. FASTER -- accelerate the ego vehicle \n \
        \t3. SLOWER -- decelerate the ego vehicle \n \
        \t4. LANE_LEFT -- change to the adjacent left lane \n \
        \t5. LANE_RIGHT -- change to the adjacent right lane\n'

        assist1 = 'I understand. Please provide the scenario: traffic conditions, speed of surrounding vehicles, ego speed, lane information, and relevant conditions so I can recommend the best action.'

        prompt2 = 'Here is the current scenario:\n \
        There are two lanes in the roundabout. The ego vehicle needs to navigate through as fast as possible while ensuring safety and avoiding collisions.\n\n'

        x, y, vx, vy = obs_[:, 1], obs_[:, 2], obs_[:, 3], obs_[:, 4]

        ego_x, ego_y = x[0], y[0]
        ego_vx, ego_vy = vx[0], vy[0]

        veh_x, veh_y = x[1:] - ego_x, y[1:] - ego_y
        veh_vx, veh_vy = vx[1:], vy[1:]

        lanes = y // 4 + 1  # Assuming lanes are divided into two groups
        ego_lane = lanes[0]
        veh_lanes = lanes[1:]

        # Ensure proper assignment for ego lanes (either lane 1 or lane 2)
        if ego_lane == 1:
            ego_left_lane = 'Left lane: Not available\n'
            ego_right_lane = 'Right lane: Lane-2\n'
        elif ego_lane == 2:
            ego_left_lane = 'Left lane: Lane-1\n'
            ego_right_lane = 'Right lane: Not available\n'
        else:
            # This shouldn't occur in a 2-lane highway scenario, but just in case
            ego_left_lane = 'Left lane: Unknown\n'
            ego_right_lane = 'Right lane: Unknown\n'

        prompt2 += 'Ego vehicle:\n \
        \tCurrent lane: Lane-' + str(ego_lane) + '\n' + '\t' + ego_left_lane + '\t' + ego_right_lane + '\tCurrent speed: ' + str(ego_vx) + ' m/s \n\n'

        lane_info = 'Lane info:\n'
        for i in range(2):
            inds = np.where(veh_lanes == i + 1)[0]
            num_v = len(inds)
            if num_v > 0:
                val, ind = self.find_smallest_positive(veh_x[inds])
                true_ind = inds[ind]
                lane_info += '\tLane-' + str(i + 1) + ': ' + str(num_v) + ' vehicle(s) ahead of ego vehicle, closest being ' + str(veh_x[true_ind]) + ' m ahead, traveling at ' + str(veh_vx[true_ind]) + ' m/s.\n'
            else:
                lane_info += '\tLane-' + str(i + 1) + ': No vehicle ahead of ego vehicle.\n'

        prompt2 += lane_info

        att_info = '\nImportant points:\n \
        \t1. SLOWER should only be used when it is critical for safety.\n \
        \t2. Lane changes should be efficient but not frequent.\n \
        \t3. Maximize speed through the roundabout without compromising safety.\n \
        \t4. Choose only one of the five listed actions: IDLE, SLOWER, FASTER, LANE_LEFT, LANE_RIGHT.\n \
        Your previous action was ' + self.prev_action + '. Based on this scenario, recommend the next action ONLY in the format \'Final decision: <final decision>\'.\n'

        prompt2 += att_info

        return prompt1, assist1, prompt2

    def step(self, action):
        obs, dqn_reward, done, truncated, info = self.env.step(action)
        self.prev_action = action_dict[action]

        Reward = 1 / (1 + np.exp(-dqn_reward))
        return obs, Reward, done, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return obs


In [24]:
vehicleCount = 10
config = {
    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": False,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 2,
    "show_trajectories": True,
    "render_agent": True,
}

# Create and configure the environment
env_roundabout = MyRoundaboutEnv_llm()

modelDqnRound = DQN.load("models/dqn_model_roundabout")

# Generate dataset from environment and LLM
observations, actions = generate_dataset(env_roundabout, modelDqnRound, "roundabout.csv")

# Train Random Forest Model
save_path2 = 'models/random_forest_model_round.pkl'
rf_model = train_rf_model(observations, actions, save_path2)


c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
Dataset Generation: 100%|██████████| 1/1 [00:52<00:00, 52.23s/it]

Dataset saved to datasets\roundabout.csv
Random Forest Model Accuracy: 100.00%
Model saved to models/random_forest_model_round.pkl


In [25]:
def rf_query_round(obs):
    ##load model
    rf_model = joblib.load("models/random_forest_model_round.pkl")
    #print(f"Model loaded")
    ##
    obs_flat = obs.flatten().reshape(1, -1)  # Flatten and reshape observation
    return rf_model.predict(obs_flat)[0]

In [26]:
class MyRoundEnvllm(gym.Env):

    def __init__(self, vehicleCount=10):
        super(MyRoundEnvllm, self).__init__()
        # base setting
        self.vehicleCount = vehicleCount
        self.prev_action  = 'FASTER'

        # environment setting
        self.config = {
            "observation": {
                "type": "Kinematics",
                "features": ["presence", "x", "y", "vx", "vy"],
                "absolute": True,
                "normalize": False,
                "vehicles_count": vehicleCount,
                "see_behind": True,
            },
            "action": {
                "type": "DiscreteMetaAction",
                "target_speeds": np.linspace(0, 32, 9),
            },
            "duration": 40,
            "vehicles_density": 2,
            "show_trajectories": True,
            "render_agent": True,
        }
        self.env = gym.make("roundabout-v0")
        self.env.configure(self.config)
        self.action_space = self.env.action_space
        self.observation_space = gym.spaces.Box(
            low=-np.inf,high=np.inf,shape=(10,5),dtype=np.float32
        )

    def find_smallest_positive(self, arr):
        smallest_positive = float('inf')
        index = -1

        for i, value in enumerate(arr):
            if 0 < value < smallest_positive:
                smallest_positive = value
                index = i

        return smallest_positive, index

    def step(self, action):

        # Step the wrapped environment and capture all returned values
        obs, dqn_reward, done, truncated, info = self.env.step(action)

        ##combine rewards randomly
        if np.random.rand() <= 0.33:

            llm_response = rf_query_round(obs)

            l_acts  = 0

            if llm_response is not None:
                l_acts += 1
                ##modify action accordingly
                llm_act = llm_response

            if l_acts == 1:
                if llm_act == action:
                    llm_reward = 1
                else:
                    llm_reward = 0
                comb_reward = 0.7*dqn_reward + 0.3*llm_reward
            else:
                comb_reward = dqn_reward

        else:
            comb_reward = dqn_reward

        self.prev_action = action_dict[action]

        Reward = 1 / (1 + np.exp(-comb_reward))

        return obs, Reward, done, truncated, info

    def reset(self, **kwargs):
        obs = self.env.reset(**kwargs)
        return obs  # Make sure to return the observation

In [27]:
from stable_baselines3.common.vec_env import DummyVecEnv
env_round_llm = MyRoundEnvllm()

c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(


In [28]:
model = DQN('MlpPolicy', env_round_llm,
            policy_kwargs=dict(net_arch=[256, 256]),
            learning_rate=5e-4,
            buffer_size=15000,
            learning_starts=200,
            batch_size=32,
            gamma=0.8,
            train_freq=1,
            gradient_steps=1,
            target_update_interval=50,
            exploration_fraction=0.7,
            verbose=1,
            tensorboard_log='highway_dqn/')

model.learn(int(2e4))
model.save('models/dqn_model_round_llm')
# Download the model to your local machine


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Logging to highway_dqn/DQN_12
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 34       |
|    ep_rew_mean      | 21.8     |
|    exploration_rate | 0.991    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 14       |
|    time_elapsed     | 9        |
|    total_timesteps  | 136      |
----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 37       |
|    ep_rew_mean      | 24.4     |
|    exploration_rate | 0.98     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 14       |
|    time_elapsed     | 20       |
|    total_timesteps  | 296      |
| train/              |          |
|    learning_rate    | 0.0005   |
|    loss             | 0.597    |
|    n_updates        | 95       |


In [29]:
from gymnasium.wrappers import RecordVideo
# base setting
vehicleCount = 10

# environment setting
config = {

    "observation": {
        "type": "Kinematics",
        "features": ["presence", "x", "y", "vx", "vy"],
        "absolute": True,
        "normalize": True,
        "vehicles_count": vehicleCount,
        "see_behind": True,
    },
    "action": {
        "type": "DiscreteMetaAction",
        "target_speeds": np.linspace(0, 32, 9),
    },
    "duration": 40,
    "vehicles_density": 2,
    "show_trajectories": True,
    "render_agent": True,
}

modelDqnRoundllm = DQN.load("models/dqn_model_round_llm")

env = gym.make('roundabout-v0', render_mode='rgb_array')
env.configure(config)
env = record_videos(env)
for episode in trange(3, desc='Test episodes'):
    (obs, info), done, truncated = env.reset(), False, False
    while not (done or truncated):
        action, _ = modelDqnRoundllm.predict(obs, deterministic=True)
        obs, reward, done, truncated, info = env.step(int(action))
env.close()
show_videos()

c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.configure to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.configure` for environment variables or `env.get_wrapper_attr('configure')` that will search the reminding wrappers.
  logger.warn(
c:\Users\dhruv\AppData\Local\Programs\Python\Python310\lib\site-packages\gymnasium\wrappers\record_video.py:94: UserWarning: WARN: Overwriting existing videos at c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(
Test episodes:   0%|          | 0/3 [00:12<?, ?it/s]

Moviepy - Building video c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-0.mp4.
Moviepy - Writing video c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-0.mp4



Test episodes:  33%|███▎      | 1/3 [00:15<00:31, 15.84s/it]

Moviepy - Done !
Moviepy - video ready c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-0.mp4


Test episodes:  33%|███▎      | 1/3 [00:27<00:31, 15.84s/it]

...

Moviepy - Done !
Moviepy - video ready c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-1.mp4


Test episodes:  67%|██████▋   | 2/3 [00:42<00:15, 15.43s/it]

Moviepy - Building video c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-2.mp4.
Moviepy - Writing video c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-2.mp4



Test episodes: 100%|██████████| 3/3 [00:46<00:00, 15.38s/it]

Moviepy - Done !
Moviepy - video ready c:\Users\dhruv\Desktop\prachit_ btp\scripts\videos\rl-video-episode-2.mp4
